In [74]:
import requests
import json
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from pathlib import Path

In [75]:
# requisicao dos dados
URL = 'https://dadosabertos.camara.leg.br/api/v2/deputados'
resposta = requests.request("GET", URL)

objetos = json.loads(resposta.text) # pega somente os dados dos deputados e descarta o resto

In [76]:
deputados = objetos['dados']    # pega os dados do deputado (lista de dicionarios)

df = pd.DataFrame(deputados)    # transforma cada dicionario em um dataFrame

df = df.drop(['id'], axis=1)    # exclui o id do deputado, pois iremos adicionar um id do banco

coluna_partido = df['siglaPartido'] # dividir em duas tabelas


In [77]:
# transformar todas as strings da siglaPartido em um id para referenciar em uma foreing key
le = LabelEncoder()
coluna_id_partido = le.fit_transform(coluna_partido)

# transforma em listas do python para poder 
coluna_id_partido_list = coluna_id_partido.tolist() # from nparray to list
coluna_partido_list = coluna_partido.values.tolist() # from pandas.series to list

In [78]:
# troca os dados do dataFrame original pelos ids das siglaPartido
df = df.replace(coluna_partido_list, coluna_id_partido_list)

# salva em um arquivo csv para exportar para o banco
filepath = Path('deputados.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath)

In [79]:
# para criar outro csv com a tabela partidos
d = dict()  # dicionario necessario para salvar sem duplicar os id e nome do partido para o csv

# cria o dicionario
for i in range(len(coluna_id_partido_list)):
    d[coluna_id_partido_list[i]] = coluna_partido[i]

# cria um dataFrame de acordo com o dicionario e organiza ele em ordem crescente pelo id
df_partido = pd.DataFrame(list(d.items()), columns=['id', 'siglaPartido']).sort_values(['id'])

# salva em um arquivo csv para exportar para o banco
filepath = Path('partidos.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_partido.to_csv(filepath, index=False)